In [11]:
#%run '/Users/zhaowenliu/Documents/Tooklits/pyuvvis/examples/Notebooks/NBCONFIG.ipynb'
#%run '/home/glue/Desktop/PYUVVIS/pyuvvis/examples/Notebooks/NBCONFIG.ipynb'
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [12]:
import re
import pyuvvis
import pyuvvis.data
import matplotlib.pyplot as plt

from datetime import datetime
from collections import OrderedDict
from pyuvvis import Spectra


from IPython.html.widgets import (
    FlexBox, VBox, HBox, HTML, Box, RadioButtons,
    FloatText, Dropdown, Checkbox, Image, 
    IntSlider, Button, Text, FloatSlider, IntText, ContainerWidget
)
from IPython.utils.traitlets import (
    link, Unicode, Float, Int, Enum, Bool, Instance, Any
)

Use [`OrderedDict`](https://docs.python.org/2/library/collections.html#collections.OrderedDict) for predictable display of key-value pairs.

In [13]:
%%html
<style>
/*
    This contents of this would go in a separate CSS file.

    Note the namespacing: this is important for two reasons.
    1) doesn't pollute the global namespace
    2) is _more specific_ than the base styles.
*/

.widget-area .spectroscopy .panel-body{
    padding: 0.5;
}
.widget-area .spectroscopy .widget-numeric-text{
    width: 5em;
}
.widget-area .spectroscopy .widget-box.start{
    margin-left: 0;
}
.widget-area .spectroscopy .widget-hslider{
    width: 12em;
}
.widget-area .spectroscopy .widget-text{
    width: 13em;
}
    
</style>

### SPECTROGRAM

In [14]:
from specgui import Box, HTML
from nbtools import mpl2html

from pyuvvis.core.spectra import _normdic as NUdic
import pyuvvis.config as pvconf
from pyuvvis.data import aunps_glass
from pyuvvis.plotting.advanced_plots import PLOTPARSER

class Spectrogram(HTML, Box):
    """
    A notional "complex widget" that knows how to redraw itself when key properties change.
    """

    # CONSTANTS (These are not traits)
    DONT_DRAW = re.compile(r'^(_.+|value|keys|comm|children|visible|parent|log|config|msg_throttle)$')
    NORMUNITS = NUdic
    NORMUNITS_REV = OrderedDict((v,k) for k,v in NORMUNITS.items())
    COLORS = ["b","g","r","y","k"]
    COLORMAPS = sorted(m for m in plt.cm.datad if not m.endswith("_r"))        
    SLIDER_STEPS = Float(25)
    
    # IO traitlets
    load_spec = Bool(False,sync=True)
    load_file = Bool(True,sync=True) #
    file_name = Unicode("", sync=True)
    save_spec = Bool(False,sync=True)
    save_spec_as = Unicode('Test',sync=True)

    # Spectra traits
    spec = Instance(Spectra)
    specname = Unicode('???', sync=True) # What should this be?
    spec_modified = Instance(Spectra)
    
    # Plotting Traits
    figwidth = Float(8)
    figheight = Float(8)
    interactive = Bool(False, sync=True)
    colorbar = Bool(False, sync=True)
    autoupdate = Bool(True, sync=True)
    colormap = Enum(COLORMAPS ,sync=True)
    color = Enum(COLORS, default_value = 'k', sync=True)
    kind = Enum(PLOTPARSER.keys(), default_value = 'spec', sync=True)

    # Units
    spec_unit = Unicode
    var_unit = Unicode 
    iunit = Unicode 
    norm_unit = Enum(NORMUNITS.values(),  sync=True)
    
    # Sampling/slicing
    slice_axis = Enum([0,1], default_value=0, sync=True)
    slice_position_start = Float(sync=True)
    slice_position_end = Float(sync=True)
    slider_start = Float(sync=True)
    slider_end = Float(sync=True)
    step_spec = Float(sync=True)
    spacing = Int(1, sync=True)
    
    selectlines = Bool(False, sync=True)
    
    
    def __init__(self, *args, **kwargs):

        # Initialize traits (_spec_changed calls initial draw)
        super(Spectrogram, self).__init__(*args, **kwargs)
        self._dom_classes += ("col-xs-9",)
        
        
    # DEFAULTS
    # --------
    def _spec_default(self):
        return getattr(pyuvvis.data, 'aunps_water')()

    def _colormap_default(self):
        return pvconf.CMAP_1DSPECPLOT #Use pyuvvis config default (red/blue map)
                
    # Events
    # ------
    def _spec_changed(self, name, old, new):
        """Overall spectrum changes; triggers most events."""

        # Leave this at this position in loop
        self.spec_modified= self.spec
        # --------------

        self._FREEZE = True #pause draws/slicing

        # Units
        self.spec_unit = self.spec.full_specunit
        self.var_unit = self.spec.full_varunit
        self.norm_unit = self.spec.full_norm
        self.iunit = self.spec.full_iunit

                
        # Spec slicing
        print self.spec.index[0], 'specindex 0'
        self.slice_position_start = self.spec.index[0]
        self.slice_position_end = self.spec.index[-1]
        self.slider_start = self.spec.index[0]
        self.slider_end = self.spec.index[-1]
        self.step_spec = (self.spec.index.max() - self.spec.index.min())/self.SLIDER_STEPS
        self.spacing = 1
      
        # Plot defaults to color map
        self._color_state = False
    
        self._FREEZE = False
        self.draw(name, old, new)
        

    def _norm_unit_changed(self, name, old, new):
        self.spec_modified = self.spec_modified.as_norm(self.NORMUNITS_REV[new])
        self.draw(name, old, new)
        
    def _iunit_changed(self, name, old, new):
        self.spec_modified.iunit = new
        # redraw?

    # Plotting events
    # ---------------
       
    def _colormap_changed(self, name, old, new):
        self._color_state = False        
        self.draw(name, old, new)
        
    def _slicing_changed(self, name, old, new):
        self.draw(name, old, new)
        
    def _color_changed(self):
        """ Because this sets colorbar, might cause 2 redraws,
        so _FREEZE used to prevent this 
        """
        self._FREEZE = True
        self.colorbar=False
        self._color_state = True
        self._FREEZE = False
        self.draw()
        
    def _colorbar_changed(self, name, old, new):
        self._color_state = False
        self.draw(name, old, new)
        
    def _interactive_changed(self, name, old, new):
        self.draw(name, old, new)
        
    # This should be phased out; plots should support colormap, area should handle accordingly
    def _kind_changed(self, name, old, new):
        self.draw(name, old, new)

    def _selectlines_changed(self, name, old, new):
        if self.interactive:
            self.draw(name, old, new)

    # IO Events
    # ---------
    # THIS SHOULD BE LOAD BUTTON CLICKED!!!!
    def _file_name_changed(self):
        try:
            self.spec = getattr(pyuvvis.data, self.file_name)()
        except AttributeError:
            pass
        
    def save_to_ns(self):
        get_ipython().user_ns[self.save_spec_as]=self.spec_modified
        print "SAVED TO: %s" % self.save_spec_as

    
    # Slicing events
    # --------------
    def _slice_position_start_changed(self, name, old, new):
        if not self._FREEZE:
            self.slice_spectrum()
            self.draw(name, old, new)

    def _slice_position_end_changed(self, name, old, new):
        if not self._FREEZE:
            self.slice_spectrum()
            self.draw(name, old, new)

    def _spacing_changed(self, name, old, new):
        """ Don't let user set less than 1 or more than dataset size"""
        # Will have to update when add var/spec slicing
        axis = self.slice_axis
        if self.spacing < 1:
            self.spacing = 1
        elif self.spacing > self.spec_modified.shape[axis]:
            self.spacing = self.spec_modified.shape[axis]
            
        self.slice_spectrum()
        self.draw(name, old, new)            
        
            
    # Draw/Slice updates
    # ------------------
    
    def slice_spectrum(self):
        # spec not spec mod, want to slice full spectra
        if self.slice_axis == 0:
            self.spec_modified = self.spec.nearby[self.slice_position_start:self.slice_position_end,::self.spacing]
        else:
            NotImplemented
#            self.spec_modified = self.spec.transpose().nearby[self.slice_position_start:self.slice_position_end:self.spacing].transpose()
            
    
    def draw(self, name=None, old=None, new=None):
        if name is not None and self.DONT_DRAW.match(name):
            return
        
        if self._FREEZE:
            return
        
        print 'in draw', name

        plot_and_message = ''

        # Add a header bubble to plot.             
       # if name is None:
       #     plot_and_message += '<div class="alert alert-info">redraw forced at %s!</div>' % (
       #         datetime.now().isoformat(' ')
       #     )
                        
        # Better way would be a decorator or something that only goes into draw if not autoupdate
        if self.autoupdate:
                        
            # Generate new figure object
            f = plt.figure(figsize=(self.figwidth, self.figheight))
            if PLOTPARSER.is_3d(self.kind):
                projection = '3d'
            else:
                projection = None
            ax = f.add_subplot(111, projection=projection)
            
            if self._color_state or self.kind not in ['spec', 'waterfall', 'contour', 'contour3d']:
                colorkwags = dict(color=self.color)
            else:
                colorkwags = dict(cmap=self.colormap, cbar=self.colorbar)
                
            self.spec_modified.plot(ax=ax, 
                  fig=f, 
                  kind=self.kind, 
                  norm=self.NORMUNITS_REV[self.norm_unit],
                  **colorkwags
                    )
            f.tight_layout() #Padding around plot
            lines = ax.get_lines()
            plt.close(f)

            #http://mpld3.github.io/modules/API.html
            if self.interactive:
                import mpld3
                if self.selectlines:
                    from line_plugin import HighlightLines
                    mpld3.plugins.connect(f, HighlightLines(lines))

                plot_and_message += mpld3.fig_to_html(f)
            else:
                plot_and_message += mpl2html(f)

            self.fig_old = f
                
        else:
            plot_and_message += html_figure(self.fig_old)

        # VALUE IS WHAT GUI LOOKS UP!!!
        self.value = plot_and_message

### RUN IT

In [15]:
from pyuvvis.data import trip_peaks, aunps_glass
from specgui import SpectralGUI

testspectra = aunps_glass().as_varunit('s')
testspectra.reference = 0

specmodel = Spectrogram(spec=testspectra) #I can initialize/set traits here
#specmodel.spacing = 10

gui = SpectralGUI(model=specmodel)
gui


430.1 specindex 0
in draw spec


in draw colormap


in draw interactive


in draw selectlines
